In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import folium
import numpy as np
import matplotlib as mpl
from folium.plugins import MarkerCluster
import json
import fiona 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.ticker as ticker
import pandas as pd
from plotly import graph_objs as go
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import pytz

In [2]:
sns.set_theme()
sns.set_palette('colorblind')

In [3]:
#import WAZE data
df=pd.read_csv('MORAalertsTotal.csv', encoding="ISO-8859-1")

In [4]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df.head()

,Id,NAME,UNIT_TYPE,Time,Alert,Sub_Alert,Street,State,Lon,Lat,Type,geometry,Park,Region,Entrance
0,40724,MORA,National Park,2017-03-05 12:54:05,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.539917,46.866887,region,POINT (-121.539917 46.866887),1,0,NaN
1,40732,MORA,National Park,2017-03-05 12:54:06,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-410,WA,-121.520425,46.869337,region,POINT (-121.520425 46.869337),1,0,Cayuse Pass
2,40734,MORA,National Park,2017-03-05 12:53:46,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.519142,46.830834,region,POINT (-121.519142 46.830834),1,0,NaN
3,40735,MORA,National Park,2017-03-05 12:54:33,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.554361,46.785824,region,POINT (-121.554361 46.785824),1,0,NaN
4,40737,MORA,National Park,2017-03-05 12:53:40,ROAD_CLOSED,ROAD_CLOSED_EVENT,SR-123,WA,-121.554361,46.785824,region,POINT (-121.554361 46.785824),1,0,NaN


In [6]:
df['Time']=pd.to_datetime(df['Time']).apply(lambda x: x.tz_localize('UTC').tz_convert('US/Pacific'))
df['Hour'] = df['Time'].dt.hour

In [7]:
df['Date'] = df['Time'].dt.date

In [8]:
df_grouped=df.groupby(['Sub_Alert']).count()
df_grouped=df_grouped.reset_index()
df_grouped=df_grouped[['Sub_Alert','Id']].sort_values(by='Sub_Alert')
df_grouped.rename(columns={'Id': 'Alerts'}, inplace=True)
df_grouped=df_grouped.sort_values(by=['Alerts'],ascending=False)
df_grouped

,Sub_Alert,Alerts
23,ROAD_CLOSED_EVENT,86310
21,JAM_STAND_STILL_TRAFFIC,7890
19,JAM_HEAVY_TRAFFIC,7232
12,HAZARD_ON_SHOULDER_CAR_STOPPED,6211
20,JAM_MODERATE_TRAFFIC,2145
4,HAZARD_ON_ROAD_CONSTRUCTION,2067
16,HAZARD_WEATHER_FOG,656
6,HAZARD_ON_ROAD_OBJECT,617
0,ACCIDENT_MAJOR,560
3,HAZARD_ON_ROAD_CAR_STOPPED,476


In [16]:
nisquallyalerts=entOnlyWOclosed.loc[entOnlyWOclosed['Entrance']=="Nisqually"]
nisquallyalerts=nisquallyalerts.loc[nisquallyalerts['Alert']=="JAM"]
df_grouped_date=nisquallyalerts.groupby(['Date']).count()
df_grouped_date=df_grouped_date.reset_index()
df_grouped_date=df_grouped_date[['Date','Id']].sort_values(by='Date')
df_grouped_date.rename(columns={'Id': 'Alerts'}, inplace=True)
df_grouped_date=df_grouped_date.sort_values(by=['Alerts'],ascending=False)
df_grouped_date

,Date,Alerts
93,2020-07-25,33
74,2019-08-03,32
101,2020-08-15,28
25,2018-05-27,24
95,2020-08-01,23
...,...,...
56,2019-05-26,1
54,2019-01-28,1
88,2020-07-03,1
122,2021-05-16,1


In [10]:
#dataset without closure alerts 
nonclosureAlerts = df.loc[df['Alert']!='ROAD_CLOSED']

In [11]:
#open hours 7:30 am - 5:00 pm

In [12]:
#park dataset with closure alerts 
parkWclosed = df.loc[df['Park']==1]

#park dataset without closure alerts 
parkWOclosed = nonclosureAlerts.loc[nonclosureAlerts['Park']==1]

In [13]:
#dataset without closure alerts  time dist
df_time_grouped=parkWOclosed.groupby(['Hour']).count()
df_time_grouped=df_time_grouped.reset_index()
df_time_grouped=df_time_grouped[['Hour','Id']].sort_values(by='Hour')
df_time_grouped

,Hour,Id
0,0,1
1,7,3
2,8,17
3,9,25
4,10,36
5,11,41
6,12,57
7,13,48
8,14,37
9,15,20


In [14]:
# entrance dataset 
entOnly=df.dropna(subset=['Entrance'])
entOnlyWOclosed=nonclosureAlerts.dropna(subset=['Entrance'])

In [15]:
#entrance dataset with closure alerts  time dist
df_time_grouped=entOnly.groupby(['Hour']).count()
df_time_grouped=df_time_grouped.reset_index()
df_time_grouped=df_time_grouped[['Hour','Id']].sort_values(by='Hour')
df_time_grouped

,Hour,Id
0,0,2689
1,1,2592
2,2,2599
3,3,2650
4,4,2657
5,5,2678
6,6,2751
7,7,2615
8,8,2768
9,9,2706


In [ ]:
#entrance dataset with closure alerts  time dist
df_time_grouped=entOnlyWOclosed.groupby(['Hour']).count()
df_time_grouped=df_time_grouped.reset_index()
df_time_grouped=df_time_grouped[['Hour','Id']].sort_values(by='Hour')
df_time_grouped

In [ ]:
# entrance analysis: time of day for all nonClosure (line graph), type of alert (set of bars for each entrance), 
    #1. comparison to RSG (exit/enter/irma)
    #2. jams by type & time of day (line graph w/ 3 lines) across all entrances 
    #3. all jams & time of day by entrance (line graph with 6 lines )
# park analysis: time of day for all nonClosure(line graph)
    #commonly jammed streets? or ranking of most jammed inside park?
    #1. jams by type & time of day (line graph w/ 3 lines)

#how to look at different alert types? -hazard from other car

# STRAVA= bike and ped analysis
    # age and time of day dist. graphs--> both on 1 graph
    # gender % diff map--> 2 separate maps
    

In [ ]:
#Adding IRMA traffic counts to # alerts dataframe without closed 
d = {'Entrance': ['Nisqually','Stevens Canyon','Cayuse Pass','SR-410 and Sunrise Rd','Mowich Lake','Carbon River'], 
     'IRMA Counts': [1087426, 433493, 472187, 771349, 553362, 111790],
    'LBS Entrance Counts': [3268, 924, 526, 3167, 283, 95],
    'LBS Exit Counts': [3417, 979, 425, 3106, 259, 77]}
IRMA = pd.DataFrame(data=d)
IRMA

In [ ]:
df_ent_grouped=entOnlyWOclosed.groupby(['Entrance']).count()
df_ent_grouped=df_ent_grouped.reset_index()
df_ent_grouped=df_ent_grouped[['Entrance','Id']].sort_values(by='Entrance')
df_ent_grouped.rename(columns={'Id': 'Alerts'}, inplace=True)

In [ ]:
df_ent_grouped

In [ ]:
d = {'Entrance': ['Carbon River'], 'Alerts': [0]}
df2 = pd.DataFrame(data=d)
df_ent_grouped=df_ent_grouped.append(df2,ignore_index=True)
df_ent_grouped

In [ ]:
wazeIrma=df_ent_grouped.join(IRMA.set_index('Entrance'), on='Entrance')
wazeIrma

In [ ]:
wazeIrma['WAZE %'] = (wazeIrma['Alerts'] / wazeIrma['Alerts'].sum())*100 
wazeIrma['IRMA %'] = (wazeIrma['IRMA Counts'] / wazeIrma['IRMA Counts'].sum())*100
wazeIrma['LBS Entrance %'] = (wazeIrma['LBS Entrance Counts'] / wazeIrma['LBS Entrance Counts'].sum())*100
wazeIrma['LBS Exit %'] = (wazeIrma['LBS Exit Counts'] / wazeIrma['LBS Exit Counts'].sum())*100
wazeIrma

In [ ]:
wazeIrma=wazeIrma.sort_values(by=['Alerts'],ascending=False)
ax = wazeIrma.plot.bar(x='Entrance',y=["WAZE %", "IRMA %","LBS Entrance %",'LBS Exit %'] )
for container, hatch in zip(ax.containers, ("//", " "," "," ")):
    for patch in container.patches:
        patch.set_hatch(hatch)
ax.legend(loc='upper right')
plt.title('Distribution of Waze Alerts Compared to IRMA and LBS Distribution by Entrance')
plt.ylabel('% of Alerts/Counts')
plt.xticks(rotation = 45)
plt.grid(False)
plt.show()

In [ ]:
entModJams=entOnlyWOclosed.loc[entOnlyWOclosed['Sub_Alert']=='JAM_MODERATE_TRAFFIC']
entHeavyJams=entOnlyWOclosed.loc[entOnlyWOclosed['Sub_Alert']=='JAM_HEAVY_TRAFFIC']
entSSJams=entOnlyWOclosed.loc[entOnlyWOclosed['Sub_Alert']=='JAM_STAND_STILL_TRAFFIC']

In [ ]:
parkModJams=parkWOclosed.loc[parkWOclosed['Sub_Alert']=='JAM_MODERATE_TRAFFIC']
parkHeavyJams=parkWOclosed.loc[parkWOclosed['Sub_Alert']=='JAM_HEAVY_TRAFFIC']
parkSSJams=parkWOclosed.loc[parkWOclosed['Sub_Alert']=='JAM_STAND_STILL_TRAFFIC']

In [ ]:
parkSSJamsGrouped=parkSSJams.groupby(['Hour']).count()
parkSSJamsGrouped=parkSSJamsGrouped.reset_index()
parkSSJamsGrouped=parkSSJamsGrouped[['Hour','Id']].sort_values(by='Hour')
parkSSJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(parkSSJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
parkSShourly_df=pd.concat([parkSSJamsGrouped , missing_df])
parkSShourly_sorted=parkSShourly_df.sort_values('Hour')
parkSShourly_sorted_df=parkSShourly_sorted.reset_index(drop=True)
parkSShourly_sorted_df

In [ ]:
entSSJamsGrouped=entSSJams.groupby(['Hour']).count()
entSSJamsGrouped=entSSJamsGrouped.reset_index()
entSSJamsGrouped=entSSJamsGrouped[['Hour','Id']].sort_values(by='Hour')
entSSJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(entSSJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
SShourly_df=pd.concat([entSSJamsGrouped , missing_df])
SShourly_sorted=SShourly_df.sort_values('Hour')
SShourly_sorted_df=SShourly_sorted.reset_index(drop=True)
SShourly_sorted_df

In [ ]:
entHeavyJamsGrouped=entHeavyJams.groupby(['Hour']).count()
entHeavyJamsGrouped=entHeavyJamsGrouped.reset_index()
entHeavyJamsGrouped=entHeavyJamsGrouped[['Hour','Id']].sort_values(by='Hour')
entHeavyJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(entHeavyJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
heavyHourly_df=pd.concat([entHeavyJamsGrouped , missing_df])
heavyHourly_sorted=heavyHourly_df.sort_values('Hour')
heavyHourly_sorted_df=heavyHourly_sorted.reset_index(drop=True)
heavyHourly_sorted_df

In [ ]:
parkHeavyJamsGrouped=parkHeavyJams.groupby(['Hour']).count()
parkHeavyJamsGrouped=parkHeavyJamsGrouped.reset_index()
parkHeavyJamsGrouped=parkHeavyJamsGrouped[['Hour','Id']].sort_values(by='Hour')
parkHeavyJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(parkHeavyJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
parkheavyHourly_df=pd.concat([parkHeavyJamsGrouped , missing_df])
parkheavyHourly_sorted=parkheavyHourly_df.sort_values('Hour')
parkheavyHourly_sorted_df=parkheavyHourly_sorted.reset_index(drop=True)
parkheavyHourly_sorted_df

In [ ]:
entModJamsGrouped=entModJams.groupby(['Hour']).count()
entModJamsGrouped=entModJamsGrouped.reset_index()
entModJamsGrouped=entModJamsGrouped[['Hour','Id']].sort_values(by='Hour')
entModJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(entModJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
modHourly_df=pd.concat([entModJamsGrouped , missing_df])
modHourly_sorted=modHourly_df.sort_values('Hour')
modHourly_sorted_df=modHourly_sorted.reset_index(drop=True)
modHourly_sorted_df

In [ ]:
parkModJamsGrouped=parkModJams.groupby(['Hour']).count()
parkModJamsGrouped=parkModJamsGrouped.reset_index()
parkModJamsGrouped=parkModJamsGrouped[['Hour','Id']].sort_values(by='Hour')
parkModJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(parkModJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
parkmodHourly_df=pd.concat([parkModJamsGrouped , missing_df])
parkmodHourly_sorted=parkmodHourly_df.sort_values('Hour')
parkmodHourly_sorted_df=parkmodHourly_sorted.reset_index(drop=True)
parkmodHourly_sorted_df

In [ ]:
plt.style.use('seaborn')
plt.plot( 'Hour', 'Alerts', data=SShourly_sorted_df, linewidth=2, label='Stand-Still')
plt.plot( 'Hour', 'Alerts', data=heavyHourly_sorted_df, linewidth=2,label='Heavy')
plt.plot( 'Hour', 'Alerts', data=modHourly_sorted_df, linewidth=2,label='Moderate')

plt.xlabel("Hour", labelpad=15)
plt.ylabel("Number of Jam Alerts", labelpad=15)
plt.title("Types of Jams by Hour at Entrances", y=1.02, fontsize=15)
plt.grid(False)
plt.legend(loc = "upper left")
plt.xlim([0, 23])
plt.ylim([0, 175])
plt.show()

In [ ]:
plt.style.use('seaborn')
plt.plot( 'Hour', 'Alerts', data=parkSShourly_sorted_df, linewidth=2, label='Stand-Still')
plt.plot( 'Hour', 'Alerts', data=parkheavyHourly_sorted_df, linewidth=2,label='Heavy')
plt.plot( 'Hour', 'Alerts', data=parkmodHourly_sorted_df, linewidth=2,label='Moderate')

plt.xlabel("Hour", labelpad=15)
plt.ylabel("Number of Jam Alerts", labelpad=15)
plt.title("Types of Jams by Hour in Park", y=1.02, fontsize=15)
plt.grid(False)
plt.xlim([0, 23])
plt.ylim([0, 175])
plt.legend(loc = "upper left")
plt.show()

In [ ]:
# make 6 dataframes=1 for each entrance only with jams --> line graph 
entJams=entOnlyWOclosed.loc[entOnlyWOclosed['Alert']=='JAM']
entJamsNisqually=entJams.loc[entJams['Entrance']=='Nisqually']
entJamsCayusePass=entJams.loc[entJams['Entrance']=='Cayuse Pass']
entJamsSR410=entJams.loc[entJams['Entrance']=='SR-410 and Sunrise Rd']
entJamsStevens=entJams.loc[entJams['Entrance']=='Stevens Canyon']
entJamsCarbon=entJams.loc[entJams['Entrance']=='Carbon River']
entJamsMowich=entJams.loc[entJams['Entrance']=='Mowich Lake']

In [ ]:
NisquallyJamsGrouped=entJamsNisqually.groupby(['Hour']).count()
NisquallyJamsGrouped=NisquallyJamsGrouped.reset_index()
NisquallyJamsGrouped=NisquallyJamsGrouped[['Hour','Id']].sort_values(by='Hour')
NisquallyJamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(0 , 24))
missing_hours = all_hours[~all_hours.isin(NisquallyJamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
NisquallyHourly_df=pd.concat([NisquallyJamsGrouped , missing_df])
NisquallyHourly_sorted=NisquallyHourly_df.sort_values('Hour')
NisquallyHourly_sorted_df=NisquallyHourly_sorted.reset_index(drop=True)
NisquallyHourly_sorted_df

In [ ]:
SR410JamsGrouped=entJamsSR410.groupby(['Hour']).count()
SR410JamsGrouped=SR410JamsGrouped.reset_index()
SR410JamsGrouped=SR410JamsGrouped[['Hour','Id']].sort_values(by='Hour')
SR410JamsGrouped.rename(columns={'Id': 'Alerts'}, inplace=True)
all_hours = pd.Series(data = range(1 , 24))
missing_hours = all_hours[~all_hours.isin(SR410JamsGrouped.Hour)]
missing_df = pd.DataFrame({'Hour' : missing_hours.values , 'Alerts' : 0})
SR410Hourly_df=pd.concat([SR410JamsGrouped , missing_df])
SR410Hourly_sorted=SR410Hourly_df.sort_values('Hour')
SR410Hourly_sorted_df=SR410Hourly_sorted.reset_index(drop=True)
SR410Hourly_sorted_df

In [ ]:
plt.style.use('seaborn')
plt.plot( 'Hour', 'Alerts', data=SR410Hourly_sorted_df, linewidth=2, label='SR-410')
plt.plot( 'Hour', 'Alerts', data=NisquallyHourly_sorted_df, linewidth=2,label='Nisqually')

plt.xlabel("Hour", labelpad=15)
plt.ylabel("Number of Jam Alerts", labelpad=15)
plt.title("Jams by Hour at Each Entrance", y=1.02, fontsize=15)
plt.grid(False)
plt.legend(loc = "upper left")
plt.show()

In [ ]:
nisqually2017=pd.read_csv('Nisqually2017.csv', encoding="ISO-8859-1")

In [ ]:
nisqually2017.head()

In [ ]:
nisqually17W=nisqually2017.loc[nisqually2017['Unnamed: 0']=='W, 1 Ch 2']
nisqually17E=nisqually2017.loc[nisqually2017['Unnamed: 0']=='E, 1 Ch 1']
nisqually17=nisqually17W.append(nisqually17E)
nisqually17

In [ ]:
nisqually17.loc['Column_Total']= nisqually17.sum(numeric_only=True, axis=0)
nisqually17

In [ ]:
nisquallyHourly=nisqually17.loc[nisqually17['Unnamed: 0']!='E, 1 Ch 1']
nisquallyHourly=nisquallyHourly.loc[nisquallyHourly['Unnamed: 0']!='W, 1 Ch 2']
nisquallyHourly

In [ ]:
nisquallyHourly['14:00']

In [ ]:
d = {'Hour': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23], 
     'Counts': [686.0,458.0,258.0,321.0,1533.0,4937.0,9561.0,12533.0,22281.0,29618.0,37381.0,43886.0,46939.0,47475.0,51931.0,55212.0,57323.0,47369.0,29992.0,19520.0,11530.0,6591.0,2839.0,1384.0]}
nisqually2017Hours = pd.DataFrame(data=d)
nisqually2017Hours

In [ ]:
plt.style.use('seaborn')
plt.plot( 'Hour', 'Counts', data=nisqually2017Hours, linewidth=2, label='Nisqually 2017 Counts')
plt.plot( 'Hour', 'Alerts', data=NisquallyHourly_sorted_df, linewidth=2,label='Nisqually Waze Alerts')

plt.xlabel("Hour", labelpad=15)
plt.ylabel("Number of Alerts/Counts", labelpad=15)
plt.title("Traffic Volumes and Alerts by Hour", y=1.02, fontsize=15)
plt.grid(False)
plt.legend(loc = "upper left")
plt.show()

In [ ]:
plt.style.use('seaborn')
x = nisqually2017Hours['Hour']
y1 = nisqually2017Hours['Counts']
y2 = NisquallyHourly_sorted_df['Alerts']

fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(x, y1, 'g-')
ax2.plot(x, y2, 'b-')

ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Traffic Counts', color='g')
ax2.set_ylabel('Jam Alerts', color='b')
plt.title("Traffic Volumes and Alerts by Hour at Nisqually", y=1.02, fontsize=15)
plt.grid(False)
plt.show()

In [ ]:
entOnlyWOclosed